# TensorFlow2 및 SMDataParallel을 사용한 분산 데이터 병렬 MNIST 훈련

## 배경
SMDataParallel은 Amazon SageMaker의 새로운 기능으로 딥러닝 모델을 더 빠르고 저렴하게 훈련할 수 있습니다. SMDataParallel은 TensorFlow2, PyTorch, MXNet을 위한 분산 데이터 병렬 훈련 프레임워크입니다.

이 노트북 예제는 MNIST 데이터셋을 사용하여 SageMaker에서 TensorFlow2와 함께 SMDataParallel을 사용하는 방법을 보여줍니다.

자세한 내용은 아래 자료들을 참조해 주세요.
1. [TensorFlow in SageMaker](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html)
2. [SMDataParallel TensorFlow API Specification](https://sagemaker.readthedocs.io/en/stable/api/training/smd_data_parallel_tensorflow.html)
3. [Getting started with SMDataParallel on SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/distributed-training.html)

**참고:** 이 예제는 SageMaker Python SDK v2.X가 필요합니다.


### 데이터셋
이 예에서는 MNIST 데이터셋을 사용합니다. MNIST는 손글씨 숫자 분류에 널리 사용되는 데이터셋으로 손으로 쓴 숫자의 70,000 개의 레이블이 있는 28x28 픽셀 그레이스케일 이미지로 구성됩니다. 데이터셋은 60,000개의 훈련 이미지와 10,000개의 테스트 이미지로 분할되며, 0~9까지의 10 개의 클래스가 존재합니다.

### SageMaker execution roles

IAM 역할 arn은 데이터에 대한 훈련 및 호스팅 액세스 권한을 부여하는 데 사용됩니다. 이를 생성하는 방법은 [Amazon SageMaker 역할](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) 을 참조하세요. 노트북 인스턴스, 훈련 및 호스팅에 둘 이상의 역할이 필요한 경우 `sagemaker.get_execution_role()`을 적절한 전체 IAM 역할 arn 문자열로 변경해 주세요.

In [ ]:
pip install sagemaker --upgrade

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

## SMDataParallel을 사용한 모델 훈련

### 훈련 스크립트

`train_tensorflow_smdataparallel_mnist.py` 스크립트는 SMDataParallel의 `DistributedGradientTape`를 사용하여 SageMaker 모델을 훈련하는 데 필요한 코드를 제공합니다. 훈련 스크립트는 SageMaker 외부에서 실행할 수있는 TensorFlow2 훈련 스크립트와 매우 유사하지만, SMDataParallel과 함께 실행되도록 수정되었습니다. SMDataParallel의 TensorFlow 클라이언트는 기본 `DistributedGradientTape`에 대한 대안을 제공합니다. 네이티브 TF2 스크립트에서 SMDataParallel을 사용하는 방법에 대한 자세한 내용은 SMDataParallel 시작하기 자습서를 참조하세요.


In [ ]:
!pygmentize code/train_tensorflow_smdataparallel_mnist.py

### SageMaker TensorFlow Estimator function options

다음 코드 블록에서 다른 인스턴스 유형, 인스턴스 수 및 분산 전략을 사용하도록 estimator 함수를 업데이트할 수 있습니다. 이전 코드 셀에서 검토한 훈련 스크립트도 estimator 함수로 전달합니다.

**인스턴스 유형**

SMDataParallel은 아래 인스턴스 유형들만 SageMaker 상에서의 모델 훈련을 지원합니다.
1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [권장]
1. ml.p4d.24xlarge [권장]

**인스턴스 수**

최상의 성능과 SMDataParallel을 최대한 활용하려면 2개 이상의 인스턴스를 사용해야 하지만, 이 예제를 테스트하는 데 1개를 사용할 수도 있습니다.

**배포 전략**

DDP 모드를 사용하려면 `distribution` 전략을 업데이트하고 `smdistributed dataparallel`을 사용하도록 설정해야 합니다.

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
                        base_job_name='tensorflow2-smdataparallel-mnist',
                        source_dir='code',
                        entry_point='train_tensorflow_smdataparallel_mnist.py',
                        role=role,
                        py_version='py37',
                        framework_version='2.3.1',
                        # For training with multinode distributed training, set this count. Example: 2
                        instance_count=2,
                        # For training with p3dn instance use - ml.p3dn.24xlarge
                        instance_type= 'ml.p3.16xlarge',
                        sagemaker_session=sagemaker_session,
                        # Training using SMDataParallel Distributed Training Framework
                        distribution={'smdistributed':{
                                            'dataparallel':{
                                                    'enabled': True
                                             }
                                      }}
                        )

In [ ]:
estimator.fit()

## Next steps

이제 훈련된 모델이 있으므로, 모델을 호스팅하는 엔드포인트를 배포할 수 있습니다. 엔드포인트를 배포한 후 추론 요청으로 테스트할 수 있습니다. 다음 셀은 추론 노트북과 함께 사용할 model_data 변수를 저장합니다.

In [ ]:
model_data = estimator.model_data
print("Storing {} as model_data".format(model_data))
%store model_data